In [4]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import pull_last_row, upload_data, pull_model

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get model from our Hopsworks model registry

In [2]:
MODEL_NAME = 'random_forest_ohlc' # this one predicts tmw_1_0_percent_increase_binary

# ***** later implement code that checks the latest model version and pulls that one
# def pull_model(model_name, model_version):
model = pull_model(MODEL_NAME, 1)

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


### Get the latest day of ethereum data

In [5]:
eth_yesterday = pull_last_row('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)

eth_yesterday

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.57s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2024-04-21 00:00:00+00:00,3142.949775,3170.679379,3137.747159,3147.137754,9.264213e+09,2.940255e+06,3.778353e+11,3214.065786,0.020823,...,0,0,0,0,0,0,0,0,0,1


### Define predictors, the input data points, and get a prediction for tmw_1_0_percent_increase_binary

In [6]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)

x0 = eth_yesterday[predictors].iloc[-1:]
x0

,ema_2,rsi_2,sma_2,last_2_day_1_0_percent_increase_count,last_2_day_1_25_percent_increase_count,last_2_day_1_5_percent_increase_count,last_2_day_1_75_percent_increase_count,last_2_day_2_0_percent_increase_count,last_2_day_2_25_percent_increase_count,last_2_day_2_5_percent_increase_count,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,1.002061,75.270475,0.998752,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
prediction = model.predict(x0)
prediction

array([0.])

In [12]:
pred = int(prediction[0])
eth_yesterday.at[eth_yesterday.index[-1], 'pred_tmw_1_0_percent_increase_binary'] = pred

eth_yesterday['pred_tmw_1_0_percent_increase_binary'] = eth_yesterday['pred_tmw_1_0_percent_increase_binary'].astype('Int64')

eth_yesterday['pred_tmw_1_0_percent_increase_binary']

0    0
Name: pred_tmw_1_0_percent_increase_binary, dtype: Int64

### Upload this prediction to a separate predictions feature group in Hopsworks

In [13]:
upload_data(eth_yesterday, 'eth_ohlc_predictions', 2)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/582805/fs/578628/fg/741743


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: eth_ohlc_predictions_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/eth_ohlc_predictions_2_offline_fg_materialization/executions
